# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,namibe,-15.1961,12.1522,22.72,70,3,3.00,AO,1715039934
1,1,port-aux-francais,-49.3500,70.2167,6.93,97,100,13.27,TF,1715039650
2,2,papatowai,-46.5619,169.4708,10.72,73,76,5.80,NZ,1715039935
3,3,puerto baquerizo moreno,-0.9000,-89.6000,23.69,88,7,3.41,EC,1715039935
4,4,klaksvik,62.2266,-6.5890,4.51,76,49,5.15,FO,1715039935


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    color = 'City')

# Display the map
map1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[city_data_df['Cloudiness'] == 0]


# Drop any rows with null values
city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,carnarvon,-24.8667,113.6333,19.60,59,0,7.45,AU,1715039936
18,18,tosno,59.5400,30.8775,1.90,78,0,1.54,RU,1715039937
29,29,mersa matruh,31.3525,27.2453,16.11,72,0,4.12,EG,1715039939
35,35,roma,43.2128,-75.4557,17.08,39,0,5.14,US,1715039736
41,41,arraial do cabo,-22.9661,-42.0278,23.98,88,0,5.66,BR,1715039664
...,...,...,...,...,...,...,...,...,...,...
520,520,chimoio,-19.1164,33.4833,16.92,100,0,0.74,MZ,1715040033
525,525,victorica,-36.2151,-65.4359,11.84,92,0,5.51,AR,1715040033
532,532,orkney,-26.9802,26.6727,15.10,26,0,1.97,ZA,1715040035
548,548,gunjur,13.2019,-16.7339,24.04,88,0,1.54,GM,1715040038


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame({
    'City': city_data_df['City'],
    'Country': city_data_df['Country'],
    'Lat': city_data_df['Lat'],
    'Lng': city_data_df['Lng'],
    'Humidity': city_data_df['Humidity'],
})
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
13,carnarvon,AU,-24.8667,113.6333,59,
18,tosno,RU,59.5400,30.8775,78,
29,mersa matruh,EG,31.3525,27.2453,72,
35,roma,US,43.2128,-75.4557,39,
41,arraial do cabo,BR,-22.9661,-42.0278,88,
...,...,...,...,...,...,...
520,chimoio,MZ,-19.1164,33.4833,100,
525,victorica,AR,-36.2151,-65.4359,92,
532,orkney,ZA,-26.9802,26.6727,26,
548,gunjur,GM,13.2019,-16.7339,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key,
    'limit':20
}

    
 # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
params["filter"] = f"circle:{102.7183},{25.0389},{radius}"
params["bias"] = f"proximity:{102.7183},{25.0389}"
    
    # Set base URL
base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
name_address = name_address.json()
name_address

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': '海韵艺术酒店',
    'country': 'China',
    'country_code': 'cn',
    'state': 'Yunnan',
    'city': 'Kunming',
    'postcode': '650051',
    'district': 'Wuhua District',
    'suburb': 'Huguo Subdistrict',
    'street': 'Qingnian Road',
    'lon': 102.7154209,
    'lat': 25.0396175,
    'formatted': 'Hai Yun Hotel, Qingnian Road, Huguo Subdistrict, 650051 Yunnan, China',
    'address_line1': 'Hai Yun Hotel',
    'address_line2': 'Qingnian Road, Huguo Subdistrict, 650051 Yunnan, China',
    'categories': ['accommodation', 'accommodation.hotel'],
    'details': ['details'],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database License',
     'url': 'https://www.openstreetmap.org/copyright',
     'raw': {'name': '海韵艺术酒店',
      'osm_id': 10672149560,
      'name:en': 'Hai Yun Hotel',
      'tourism': 'hotel',
      'osm_type': 

In [23]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key,
    'limit':20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
carnarvon - nearest hotel: No hotel found
tosno - nearest hotel: Комфорт
mersa matruh - nearest hotel: صقور
roma - nearest hotel: Wingate by Wyndham Rome
arraial do cabo - nearest hotel: No hotel found
port augusta - nearest hotel: Hotel Commonwealth
al kharijah - nearest hotel: Kharga Hotel
anadyr - nearest hotel: Гостевой дом
port lincoln - nearest hotel: Boston Hotel
carthage - nearest hotel: Hôtel Amilcar
sakakah - nearest hotel: فندق النزل
hami - nearest hotel: Хами
naantali - nearest hotel: Hotel Bridget Inn
ghazni - nearest hotel: No hotel found
luderitz - nearest hotel: Seaview Hotel Zum Sperrgebiet
am timan - nearest hotel: Nimla
hun - nearest hotel: فندق الهروج
purna - nearest hotel: No hotel found
ilhabela - nearest hotel: Vila Kebaya
khanty-mansiysk - nearest hotel: Гамма
pisco - nearest hotel: La Portada
bredasdorp - nearest hotel: Victoria Hotel
inebolu - nearest hotel: osman sungur
tsiombe - nearest hotel: No hotel found
holmen - nearest hotel: Prai

,City,Country,Lat,Lng,Humidity,Hotel Name
13,carnarvon,AU,-24.8667,113.6333,59,No hotel found
18,tosno,RU,59.5400,30.8775,78,Комфорт
29,mersa matruh,EG,31.3525,27.2453,72,صقور
35,roma,US,43.2128,-75.4557,39,Wingate by Wyndham Rome
41,arraial do cabo,BR,-22.9661,-42.0278,88,No hotel found
...,...,...,...,...,...,...
520,chimoio,MZ,-19.1164,33.4833,100,Hotel Amirana
525,victorica,AR,-36.2151,-65.4359,92,Hotel Mazamet
532,orkney,ZA,-26.9802,26.6727,26,No hotel found
548,gunjur,GM,13.2019,-16.7339,88,Footsteps Eco Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Configure the map plot
map2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    color = 'City',
    hover_cols = ['Hotel Name','Country']
)

# Display the map
map2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)